In [1]:
import os
import wandb
import torch
import torch.nn as nn
import random
from torch.autograd import Variable
from torch.utils.data import DataLoader
import pandas as pd
import torch.optim as optim
import torch.nn.functional as Function
import argparse


In [2]:
SYMBOL_BEGIN, SYMBOL_END, SYMBOL_UNKNOWN, SYMBOL_PADDING = 0, 1, 2, 3

INPUT_LABEL = "input"
TARGET_LABEL = "target"
DELIMETER = ","

RNN_KEY = "RNN"
GRU_KEY = "GRU"
LSTM_KEY = "LSTM"

INPUT_LANG_KEY = "input_lang"
OUTPUT_LANG_KEY = "output_lang"
PAIRS_KEY = "pairs"
MAX_LEN_KEY = "max_len"

input_lang = "eng"
TARGET_LANG = "hin"

TRAIN_LABEL = "train"
TEST_LABEL = "test"
VALID_LABEL = "valid"

DEFAULT_PATH = "/kaggle/input/aksharantar-sampled/aksharantar_sampled"
TRAIN_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{TRAIN_LABEL}.csv"
VALIDATION_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{VALID_LABEL}.csv"
TEST_DATASET_PATH = f"{DEFAULT_PATH}/{TARGET_LANG}/{TARGET_LANG}_{TEST_LABEL}.csv"

NADAM_KEY = "Nadam"

# Sweep param labels
EMBEDDING_SIZE_KEY = "embedding_size"
EPOCHS_KEY = "epochs"
ENCODER_LAYER_KEY = "encoder_layers"
DECODER_LAYER_KEY = "decoder_layers"
HIDDEN_LAYER_KEY = "hidden_layer"
IS_BIDIRECTIONAL_KEY = "bidirectional"
DROPOUT_KEY = "dropout"
CELL_TYPE_KEY = "cell_type"
LEARNING_RATE_KEY = "learning_rate"
BATCH_SIZE_KEY = "batch_size"

# wandb constants
WANDB_PROJECT_NAME="dl-assignment-3"
WANDB_ENTITY_NAME="cs23m007"

# wandb plot titles
TRAIN_ACCURACY_TITLE = "train_acc"
VALIDATION_ACCURACY_TITLE = "val_acc"
TEST_ACCURACY_TITLE = "test_acc"
TRAIN_LOSS_TITLE = "train_loss"
VALIDATION_LOSS_TITLE = "val_loss"
TEST_LOSS_TITLE = "test_loss"

best_params = {
    EMBEDDING_SIZE_KEY :256,
    EPOCHS_KEY :5,
    ENCODER_LAYER_KEY :2,
    DECODER_LAYER_KEY :2,
    HIDDEN_LAYER_KEY :256,
    IS_BIDIRECTIONAL_KEY :False,
    DROPOUT_KEY :0.2,
    CELL_TYPE_KEY :LSTM_KEY,
    BATCH_SIZE_KEY : 32,
    LEARNING_RATE_KEY: 0.001

}




# Set the device type to CUDA if available, otherwise use CPU
is_gpu = torch.cuda.is_available()
if is_gpu:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")


In [ ]:
sweep_config = {
    "name" : "CS6910_Assignemnt3_Without Attention",
    "method" : "random",
    'metric': {
        'name': VALIDATION_ACCURACY_TITLE,
        'goal': 'maximize'
    },
    "parameters" : {
        EMBEDDING_SIZE_KEY : {
          "values" : [16, 32, 64, 256]  
        },
        EPOCHS_KEY : {
            "values" : [5,10]
        },
        ENCODER_LAYER_KEY: {
            "values": [1,2,3]
        },
        DECODER_LAYER_KEY: {
            "values": [1,2,3]
        },
        HIDDEN_LAYER_KEY:{
            "values": [16, 32, 64, 256]
        },
        IS_BIDIRECTIONAL_KEY:{
            "values": [True, False]
        },
        DROPOUT_KEY: {
            "values": [0,0.2,0.3]       
        }, 
        CELL_TYPE_KEY: {
            "values": [RNN_KEY,GRU_KEY,LSTM_KEY]       
        },
        LEARNING_RATE_KEY:{
            "values":[0.001,0.01]
        },
        BATCH_SIZE_KEY:{
            "values":[32,64,128] 
        }
    }
}

# Utility Functions and classes

In [3]:
class Vocabulary:
    def __init__(self):
        self.str_count,self.int_encodding = dict(),dict()
        self.n_chars = 4
        self.str_encodding = {0: "<", 1: ">", 2: "?", 3: "."}

    def addWord(self, word):
        for char in word:
            try:
                self.str_count[char] += 1
            except:
                self.int_encodding[char] = self.n_chars
                self.str_encodding[self.n_chars] = char
                self.str_count[char] = 1
                self.n_chars += 1

# prepareDataWithoutAttn
def prepareData(dir):
    data = pd.read_csv(dir, sep=DELIMETER, names=[INPUT_LABEL, TARGET_LABEL])

    max_input_length = data[INPUT_LABEL].apply(len).max()
    max_target_length = data[TARGET_LABEL].apply(len).max()
    
    max_len=max(max_input_length,max_target_length)

    input_lang, output_lang = Vocabulary(), Vocabulary()

    pairs = pd.concat([data[INPUT_LABEL], data[TARGET_LABEL]], axis=1).values.tolist()

    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])

    return input_lang,output_lang,pairs,max_len



def prepareDataWithAttention(dir):

    data = pd.read_csv(dir, sep=DELIMETER, names=[INPUT_LABEL, TARGET_LABEL])

    max_input_length = data[INPUT_LABEL].apply(len).max()
    max_target_length = data[TARGET_LABEL].apply(len).max()
    

    input_lang, output_lang = Vocabulary(), Vocabulary()
    

    pairs = pd.concat([data[INPUT_LABEL], data[TARGET_LABEL]], axis=1).values.tolist()

    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])

    prepared_data = {
        "input_lang": input_lang,
        "output_lang": output_lang,
        "pairs": pairs,
        "max_input_length": max_input_length,
        "max_target_length": max_target_length,
    }

    return prepared_data



# helpTensorWithoutAttn
def helpTensor(lang, word, max_length):
    index_list = []
    for char in word:
        try:
            index_list.append(lang.char2index[char])
        except:
            index_list.append(SYMBOL_UNKNOWN)

    indexes = index_list
    indexes.append(SYMBOL_END)
    n = len(indexes)
    indexes.extend([SYMBOL_PADDING] * (max_length - n))
    result = torch.LongTensor(indexes)
    if is_gpu:
        return result.cuda()
    return result


def helpTensorWithAttention(lang, word, max_length):
    index_list=[]
    for char in word:
        try:
            index_list.append(lang.char2index[char])
        except:
            index_list.append(SYMBOL_UNKNOWN)
    indexes = index_list
    indexes.append(SYMBOL_END)
    n = len(indexes)
    indexes.extend([SYMBOL_PADDING] * (max_length - n))
    result = torch.LongTensor(indexes)
    if is_gpu:
        return result.cuda()
    return result


# MakeTensorWithoutAttn
def makeTensor(input_lang, output_lang, pairs, reach):
    res = [(helpTensor(input_lang, pairs[i][0], reach), helpTensor(output_lang, pairs[i][1], reach)) for i in range(len(pairs))]
    return res

# accuracyWithoutAttn
def accuracy(encoder, decoder, loader, batch_size, criterion, cell_type, num_layers_enc, max_length, output_lang):
    with torch.no_grad():
        total = correct = 0

        for batch_x, batch_y in loader:
            encoder_hidden = encoder.initHidden(batch_size, num_layers_enc)

            input_variable = Variable(batch_x.transpose(0, 1))
            target_variable = Variable(batch_y.transpose(0, 1))

            if cell_type == LSTM_KEY:
                encoder_cell_state = encoder.initHidden(batch_size, num_layers_enc)
                encoder_hidden = (encoder_hidden, encoder_cell_state)


            output = torch.LongTensor(target_variable.size()[0], batch_size)

            for ei in range(input_variable.size()[0]):
                encoder_hidden = encoder(input_variable[ei], batch_size, encoder_hidden)[1]

            decoder_input = Variable(torch.LongTensor([SYMBOL_BEGIN] * batch_size))
            if is_gpu:
                decoder_input = decoder_input.cuda()

            decoder_hidden = encoder_hidden

            # Decoder forward pass
            for di in range(target_variable.size()[0]):
                decoder_output, decoder_hidden = decoder(decoder_input, batch_size, decoder_hidden)
                topi = decoder_output.data.topk(1)[1]
                output[di], decoder_input = torch.cat(tuple(topi)), torch.cat(tuple(topi))
            output = output.transpose(0, 1)

            # Calculate accuracyWithoutAttn
            for di in range(output.size()[0]):
                ignore = [SYMBOL_BEGIN, SYMBOL_END, SYMBOL_PADDING]
                sent = [output_lang.str_encodding[letter.item()] for letter in output[di] if letter not in ignore]
                y = [output_lang.str_encodding[letter.item()] for letter in batch_y[di] if letter not in ignore]
                if sent == y:
                    correct += 1
                total += 1

    return (correct / total) * 100

def accuracyWithAttention(
    encoder,
    decoder,
    loader,
    batch_size,
    num_layers_enc,
    cell_type,
    output_lang,
    criterion,
    max_length,
    ):

    with torch.no_grad():

        total = correct = 0

        for batch_x, batch_y in loader:

            encoder_hidden = encoder.initHidden(batch_size, num_layers_enc)

            input_variable = Variable(batch_x.transpose(0, 1))
            target_variable = Variable(batch_y.transpose(0, 1))

            if cell_type == LSTM_KEY:
                encoder_cell_state = encoder.initHidden(batch_size, num_layers_enc)
                encoder_hidden = (encoder_hidden, encoder_cell_state)


            output = torch.LongTensor(target_variable.size()[0], batch_size)

            encoder_outputs = Variable(
                torch.zeros(max_length, batch_size, encoder.hidden_size)
            )
            if is_gpu: 
                encoder_outputs = encoder_outputs.cuda() 

            for ei in range(input_variable.size()[0]):
                encoder_output, encoder_hidden = encoder(
                    input_variable[ei], batch_size, encoder_hidden
                )
                encoder_outputs[ei] = encoder_output[0]

            decoder_input = Variable(torch.LongTensor([SYMBOL_BEGIN] * batch_size))
            if is_gpu:
                decoder_input = decoder_input.cuda() 

            decoder_hidden = encoder_hidden

            for di in range(target_variable.size()[0]):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input,
                    batch_size,
                    decoder_hidden,
                    encoder_outputs.reshape(
                        batch_size, max_length, encoder.hidden_size
                    ),
                )
                topi = decoder_output.data.topk(1)[1]
                decoder_input,output[di] = torch.cat(tuple(topi)),torch.cat(tuple(topi))
            output = output.transpose(0, 1)

            for di in range(output.size()[0]):
                ignore = [SYMBOL_BEGIN, SYMBOL_END, SYMBOL_PADDING]
                sent = [output_lang.str_encodding[letter.item()] for letter in output[di] if letter not in ignore]
                y = [output_lang.str_encodding[letter.item()] for letter in batch_y[di] if letter not in ignore]
                if sent == y:
                    correct += 1
                total += 1

    return (correct / total) * 100


# calc_lossWithoutAttn
def calc_loss(encoder, decoder, input_tensor, target_tensor, batch_size, encoder_optimizer, decoder_optimizer, criterion, cell_type, num_layers_enc, max_length, is_training, teacher_forcing_ratio=0.5):
    # Initialize the encoder hidden state
    output_hidden = encoder.initHidden(batch_size, num_layers_enc)

    # Check if LSTM and initialize cell state
    if cell_type == LSTM_KEY:
        encoder_cell_state = encoder.initHidden(batch_size, num_layers_enc)
        output_hidden = (output_hidden, encoder_cell_state)

    # Zero the gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Get input and target sequence lengths
    # input_length = input_tensor.size(0)
    # target_length = target_tensor.size(0)

    # Initialize loss
    loss = 0

    # Encoder forward pass
    for ei in range(input_tensor.size(0)):
        output_hidden = encoder(input_tensor[ei], batch_size, output_hidden)[1]

    # Initialize decoder input
    decoder_input = torch.LongTensor([SYMBOL_BEGIN] * batch_size)
    decoder_input = decoder_input.cuda() if is_gpu else decoder_input

    # Determine if using teacher forcing
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Loop over target sequence
    if is_training:
        # Training phase
        for di in range(target_tensor.size(0)):
            decoder_output, output_hidden = decoder(decoder_input, batch_size, output_hidden)
            decoder_input = target_tensor[di] if use_teacher_forcing else decoder_output.argmax(dim=1)
            loss = criterion(decoder_output, target_tensor[di]) + loss
    else:
        # Validation phase
        with torch.no_grad():
            for di in range(target_tensor.size(0)):
                decoder_output, output_hidden = decoder(decoder_input, batch_size, output_hidden)
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = decoder_output.argmax(dim=1)

    # Backpropagation and optimization in training phase
    if is_training:
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

    # Return the average loss per target length
    return loss.item() / target_tensor.size(0)

def calcLossWithAttention(
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    input_tensor,
    target_tensor,
    criterion,
    batch_size,
    cell_type,
    num_layers_enc,
    max_length,is_training,
    teacher_forcing_ratio=0.5,
):

    output_hidden = encoder.initHidden(batch_size, num_layers_enc)

    if cell_type == LSTM_KEY:
        encoder_cell_state = encoder.initHidden(batch_size, num_layers_enc)
        output_hidden = (output_hidden, encoder_cell_state)

    if is_training:
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = Variable(torch.zeros(max_length, batch_size, encoder.hidden_size))
    if is_gpu:
        encoder_outputs = encoder_outputs.cuda()
    loss = 0

    for ei in range(input_length):
        encoder_output, output_hidden = encoder(
            input_tensor[ei], batch_size, output_hidden
        )
        encoder_outputs[ei] = encoder_output[0]

    decoder_input = Variable(torch.LongTensor([SYMBOL_BEGIN] * batch_size))
    if is_gpu :
        decoder_input = decoder_input.cuda() 
    decoder_hidden = output_hidden
    if is_training:
        use_teacher_forcing = False
        if random.random() < teacher_forcing_ratio:
            use_teacher_forcing = True
        n = target_length
        if not use_teacher_forcing :
            for di in range(n):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input,
                    batch_size,
                    decoder_hidden,
                    encoder_outputs.reshape(batch_size, max_length, encoder.hidden_size),
                )
                #2 for loop ko bhar dal de
                topv, topi = decoder_output.data.topk(1)
                decoder_input = torch.cat(tuple(topi))
                if is_gpu :
                    decoder_input = decoder_input.cuda() 
                loss += criterion(decoder_output, target_tensor[di])
        else:
            for di in range(n):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input,
                    batch_size,
                    decoder_hidden,
                    encoder_outputs.reshape(batch_size, max_length, encoder.hidden_size),
                )
                loss += criterion(decoder_output, target_tensor[di])
                decoder_input = target_tensor[di]
            

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()
    else :
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input,
                batch_size,
                decoder_hidden,
                encoder_outputs.reshape(batch_size, max_length, encoder.hidden_size),
            )
            topv, topi = decoder_output.data.topk(1)
            decoder_input = torch.cat(tuple(topi))

            if is_gpu:
                decoder_input = decoder_input.cuda()
            loss += criterion(decoder_output, target_tensor[di])

    avg_loss = loss.item() / target_length
    return avg_loss




# Train and evaluate the Seq2SeqWithoutAttn model
def seq2seq(encoder, decoder, train_loader, val_loader, test_loader, lr, optimizer, epochs, max_length_word, num_layers_enc, output_lang,batch_size,cell_type,is_wandb):
    max_length = max_length_word - 1
    # Define the optimizer and criterion
    encoder_optimizer = optim.NAdam(encoder.parameters(), lr=lr) if optimizer == "nadam" else optim.Adam(encoder.parameters(), lr=lr)
    decoder_optimizer = optim.NAdam(decoder.parameters(), lr=lr) if optimizer == "nadam" else optim.Adam(decoder.parameters(), lr=lr)
    criterion = nn.NLLLoss()

    for epoch in range(epochs):
        train_loss_total = 0
        val_loss_total = 0

        # Training phase
        for batch_x, batch_y in train_loader:
            batch_x = Variable(batch_x.transpose(0, 1))
            batch_y = Variable(batch_y.transpose(0, 1))
            # Calculate the training loss
            loss = calc_loss(encoder, decoder, batch_x, batch_y, batch_size, encoder_optimizer, decoder_optimizer, criterion, cell_type, num_layers_enc, max_length, is_training=True)
            train_loss_total += loss

        train_loss_avg = train_loss_total / len(train_loader)
        print(f"Epoch: {epoch} | Train Loss: {train_loss_avg:.4f} |", end="")

        # Validation phase
        for batch_x, batch_y in val_loader:
            batch_x = Variable(batch_x.transpose(0, 1))
            batch_y = Variable(batch_y.transpose(0, 1))
            # Calculate the validation loss
            loss = calc_loss(encoder, decoder, batch_x, batch_y, batch_size, encoder_optimizer, decoder_optimizer, criterion, cell_type, num_layers_enc, max_length, is_training=False)
            val_loss_total += loss

        val_loss_avg = val_loss_total / len(val_loader)
        print(f"Val Loss: {val_loss_avg:.4f} |", end="")


        train_acc = accuracy(encoder, decoder, train_loader, batch_size, criterion, cell_type, num_layers_enc, max_length, output_lang)
        train_acc /= 100
        print(f"train Accuracy: {train_acc:.4%} |", end="")

        # Calculate validation accuracyWithoutAttn
        val_acc = accuracy(encoder, decoder, val_loader, batch_size, criterion, cell_type, num_layers_enc, max_length, output_lang)
        val_acc /= 100
        print(f"Val Accuracy: {val_acc:.4%} |", end="")
        
        test_acc = accuracy(encoder, decoder, test_loader, batch_size, criterion, cell_type, num_layers_enc, max_length, output_lang)
        test_acc /= 100
        print(f"Test Accuracy: {test_acc:.4%}")
        if is_wandb:
            wandb.log(
                {
                    TRAIN_ACCURACY_TITLE: train_acc,
                    VALIDATION_ACCURACY_TITLE: val_acc,
                    TEST_ACCURACY_TITLE: test_acc,
                    TRAIN_LOSS_TITLE: train_loss_avg,
                    VALIDATION_LOSS_TITLE: val_loss_avg,
                    # TEST_LOSS_TITLE: test_loss
                }
            )


def seq2seqWithAttention(
    encoder,
    decoder,
    train_loader,
    val_loader,
    test_loader,
    learning_rate,
    optimizer,
    epochs,
    max_length_word,
    attention,
    num_layers_enc,
    output_lang,
    batch_size,
    cell_type,
    is_wandb
 ):
    max_length = max_length_word - 1
    n_val = len(val_loader)
    n_train = len(train_loader)
    encoder_optimizer = (
        optim.NAdam(encoder.parameters(), lr=learning_rate)
        if optimizer == "nadam"
        else optim.Adam(encoder.parameters(), lr=learning_rate)
    )
    decoder_optimizer = (
        optim.NAdam(decoder.parameters(), lr=learning_rate)
        if optimizer == "nadam"
        else optim.Adam(decoder.parameters(), lr=learning_rate)
    )
    criterion = nn.NLLLoss()

    for epoch in range(epochs):
        train_loss_total, val_loss_total = 0, 0
        
        for batchx, batchy in train_loader:
            batchx = Variable(batchx.transpose(0, 1))
            batchy = Variable(batchy.transpose(0, 1))
            loss = calcLossWithAttention(
                encoder = encoder,
                decoder =decoder,
                encoder_optimizer = encoder_optimizer,
                decoder_optimizer = decoder_optimizer,
                input_tensor = batchx,
                target_tensor = batchy,
                criterion = criterion,
                batch_size =  batch_size,
                cell_type = cell_type,
                num_layers_enc = num_layers_enc,
                max_length = max_length + 1,
                is_training = True, #is_training
                teacher_forcing_ratio=0.5
            )
            train_loss_total = train_loss_total + loss
        train_loss_avg = train_loss_total / n_train
        print(f"Epoch: {epoch} | Train Loss: {train_loss_avg:.4f} | ", end="")

        for batchx, batchy in val_loader:
            batchx = Variable(batchx.transpose(0, 1))
            batchy = Variable(batchy.transpose(0, 1))
            loss = calcLossWithAttention(
                encoder = encoder,
                decoder = decoder,
                encoder_optimizer = encoder_optimizer,
                decoder_optimizer = decoder_optimizer,
                input_tensor = batchx,
                target_tensor = batchy,
                criterion = criterion,
                batch_size =  batch_size,
                cell_type = cell_type,
                num_layers_enc = num_layers_enc,
                max_length = max_length + 1,
                is_training = False, #is_training
                teacher_forcing_ratio=0.5
            )

            val_loss_total = val_loss_total+ loss

        val_loss_avg = val_loss_total / n_val
        print(f"Val Loss: {val_loss_avg:.4f} | ", end="")

        train_acc = accuracyWithAttention(
            encoder = encoder,
            decoder = decoder,
            loader = train_loader,
            batch_size = batch_size,
            num_layers_enc = num_layers_enc,
            cell_type = cell_type,
            output_lang = output_lang,
            criterion = criterion,
            max_length = max_length,
        ) 
        train_acc = train_acc /  100
        print(f"Train Accuracy: {train_acc:.4%} |", end="")

        val_acc = accuracyWithAttention(
            encoder = encoder,
            decoder = decoder,
            loader = val_loader,
            batch_size = batch_size,
            num_layers_enc = num_layers_enc,
            cell_type = cell_type,
            output_lang = output_lang,
            criterion = criterion,
            max_length = max_length + 1,
        )
        val_acc = val_acc / 100
        print(f"Val Accuracy: {val_acc:.4%} |", end="")
        test_acc = accuracyWithAttention(
            encoder = encoder,
            decoder = decoder,
            loader = test_loader,
            batch_size = batch_size,
            num_layers_enc = num_layers_enc,
            cell_type = cell_type,
            output_lang = output_lang,
            criterion = criterion,
            max_length = max_length + 1,
        )

        test_acc = test_acc / 100
        print(f"Test Accuracy: {test_acc:.4%}")

        if is_wandb:
            wandb.log(
                {
                    TRAIN_ACCURACY_TITLE: train_acc,
                    VALIDATION_ACCURACY_TITLE: val_acc,
                    TEST_ACCURACY_TITLE: test_acc,
                    TRAIN_LOSS_TITLE: train_loss_avg,
                    VALIDATION_LOSS_TITLE: val_loss_avg,
                    # TEST_LOSS_TITLE: test_loss
                }
            )



 


# Encoder Class

In [6]:
# EncoderRNNWithoutAttn
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers_encoder, cell_type, drop_out, bi_directional):
        super(EncoderRNN, self).__init__()

        self.emb_n = embedding_size
        self.hid_n = hidden_size
        self.encoder_n = num_layers_encoder
        self.model_key = cell_type
        self.is_dropout = drop_out
        self.is_bi_dir = bi_directional

        self.embedding = nn.Embedding(input_size, self.emb_n)
        self.dropout = nn.Dropout(self.is_dropout)

        cell_map = dict({RNN_KEY: nn.RNN, GRU_KEY: nn.GRU, LSTM_KEY: nn.LSTM})
        self.cell_layer = cell_map[self.model_key](
            input_size = self.emb_n,
            hidden_size = self.hid_n,
            num_layers=self.encoder_n,
            dropout=self.is_dropout,
            bidirectional=self.is_bi_dir,
        )

    def forward(self, input, batch_size, hidden):
        pre_embedded = self.embedding(input)
        transformed_embedded_data = pre_embedded.view(1, batch_size, -1)
        embedded = self.dropout(transformed_embedded_data)
        y_cap, hidden = self.cell_layer(embedded, hidden)
        return y_cap, hidden

    def initHidden(self, batch_size, num_layers_enc):
        if self.is_bi_dir:
            weights = torch.zeros(num_layers_enc * 2 , batch_size, self.hid_n)
        else:
            weights = torch.zeros(num_layers_enc, batch_size, self.hid_n)

        if is_gpu:
            return weights.cuda()
        return weights
    

class EncoderRNNWithAttention(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size,num_layers_encoder,cell_type,drop_out,bi_directional):
        super(EncoderRNNWithAttention, self).__init__()

        self.emb_n = embedding_size
        self.hid_n = hidden_size
        self.encoder_n = num_layers_encoder
        self.model_key = cell_type
        self.is_dropout = drop_out
        self.is_bi_dir = bi_directional

        self.embedding = nn.Embedding(input_size, self.emb_n)
        self.dropout = nn.Dropout(self.is_dropout)

        cell_map = dict({RNN_KEY: nn.RNN, GRU_KEY: nn.GRU, LSTM_KEY: nn.LSTM})
        self.cell_layer = cell_map[self.model_key](
            input_size = self.emb_n,
            hidden_size = self.hid_n,
            num_layers=self.encoder_n,
            dropout=self.is_dropout,
            bidirectional=self.is_bi_dir,
        )

    def forward(self, input, batch_size, hidden):
        pre_embedded = self.embedding(input)
        transformed_embedded_data = pre_embedded.view(1, batch_size, -1)
        embedded = self.dropout(transformed_embedded_data)
        y_cap, hidden = self.cell_layer(embedded, hidden)
        return y_cap, hidden

    def initHidden(self, batch_size, num_layers_enc):
        if self.is_bi_dir:
            weights = torch.zeros(num_layers_enc * 2 , batch_size, self.hid_n)
        else:
            weights = torch.zeros(num_layers_enc, batch_size, self.hid_n)

        if is_gpu:
            return weights.cuda()
        return weights
 



# Decoder Class

In [ ]:
# DecoderRNNWithoutAttn
class DecoderRNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, num_layers_decoder, cell_type, drop_out, bi_directional, output_size):
        super(DecoderRNN, self).__init__()

        self.emb_n = embedding_size
        self.hid_n = hidden_size
        self.decoder_n = num_layers_decoder
        self.model_key = cell_type
        self.is_dropout = drop_out
        self.is_bi_dir = bi_directional

        # Create an embedding layer
        self.embedding = nn.Embedding(output_size, self.emb_n)
        self.dropout = nn.Dropout(self.is_dropout)

        cell_map = dict({RNN_KEY: nn.RNN, GRU_KEY: nn.GRU, LSTM_KEY: nn.LSTM})

        if self.cell_type in cell_map:
            self.cell_layer = cell_map[self.model_key](
                input_size = self.emb_n,
                hidden_size = self.hid_n,
                num_layers=self.decoder_n,
                dropout=self.is_dropout,
                bidirectional=self.is_bi_dir,
            )

        # Linear layer for output
        if self.is_bi_dir :
            self.out = nn.Linear(self.hid_n * 2, output_size)
        else:
            self.out = nn.Linear(self.hid_n,output_size)

        # Softmax activation
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, batch_size, hidden):
        y_cap = Function.relu(self.dropout(self.embedding(input).view(1, batch_size, -1)))
        y_cap, hidden = self.cell_layer(y_cap, hidden)

        y_cap = self.softmax(self.out(y_cap[0]))
        return y_cap, hidden


class DecoderRNNWithAttention(nn.Module):
    def __init__(
        self,
        hidden_size,
        embedding_size,
        cell_type,
        num_layers_decoder,
        drop_out,
        max_length_word,
        output_size,
    ):

        super(DecoderRNNWithAttention, self).__init__()

        self.hid_n = hidden_size
        self.emb_n = embedding_size
        self.model_key = cell_type
        self.decoder_n = num_layers_decoder
        self.drop_out = drop_out
        self.max_length_word = max_length_word

        self.embedding = nn.Embedding(output_size, embedding_dim=self.emb_n)
        layer_size = self.emb_n + self.hid_n 
        self.attention_layer = nn.Linear(
            layer_size , self.max_length_word
        )
        self.attention_combine = nn.Linear(
            layer_size, self.emb_n
        )
        self.dropout = nn.Dropout(self.drop_out)

        self.cell_layer = None
        cell_map = dict({RNN_KEY: nn.RNN, GRU_KEY: nn.GRU, LSTM_KEY: nn.LSTM})

        if self.model_key in cell_map:
            self.cell_layer = cell_map[self.model_key](
                self.emb_n,
                self.hid_n,
                num_layers=self.decoder_n,
                dropout=self.drop_out,
            )

        self.out = nn.Linear(self.hid_n, output_size)

    def forward(self, input, batch_size, hidden, encoder_outputs):
        pre_embedded = self.embedding(input)
        embedded = pre_embedded.view(1, batch_size, -1)

        attention_weights = None

        attention_weights = Function.softmax(
            self.attention_layer(torch.cat((embedded[0],hidden[0][0] if self.model_key != LSTM_KEY else  hidden[0]), 1)), dim=1
        )

        attention_applied = torch.bmm(
            attention_weights.view(batch_size, 1, self.max_length_word),
            encoder_outputs,
        ).view(1, batch_size, -1)

        y_cap = torch.cat((embedded[0], attention_applied[0]), 1)
        y_cap = Function.relu(self.attention_combine(y_cap).unsqueeze(0))
        # if self.cell_type=RNN" :
        y_cap, hidden = self.cell_layer(y_cap, hidden)
        y_cap = Function.log_softmax(self.out(y_cap[0]), dim=1)

        return y_cap, hidden, attention_weights




# Train Function

In [ ]:
def train(config_defaults = best_params,flag = False,is_wandb = True):
    optimizer = NADAM_KEY
    if flag:
        pass
    else:
        # Prepare training data
        if is_wandb:
            wandb.init(project=WANDB_PROJECT_NAME, entity=WANDB_ENTITY_NAME,config = config_defaults)
            args = wandb.config
            # Set the name of the run

            wandb.run.name = 'ep-'+str(args[EPOCHS_KEY])+'-lr-'+str(args[LEARNING_RATE_KEY])+'-bs-'+str(args[BATCH_SIZE_KEY])+'-el-'+str(args[ENCODER_LAYER_KEY])+'-dl-'+str(args[DECODER_LAYER_KEY]) \
                            +'-hl-'+str(args[HIDDEN_LAYER_KEY])+'-do-'+ str(args[DROPOUT_KEY])+ '-es-'+str(args[EMBEDDING_SIZE_KEY]) \
                            + '-is_bd-'+str(args[IS_BIDIRECTIONAL_KEY])+'-model'+str(args[CELL_TYPE_KEY])

        input_langs,output_langs,pairs,max_len = prepareData(TRAIN_DATASET_PATH)
        print("train:sample:", random.choice(pairs))
        train_n = len(pairs)
        print(f"Number of training examples: {train_n}")

        # Prepare validation data
        input_langs,output_langs,val_pairs,max_len_val = prepareData(VALIDATION_DATASET_PATH)
        val_n = len(val_pairs)
        print("validation:sample:", random.choice(val_pairs))
        print(f"Number of validation examples: {val_n}")

        # Prepare test data
        input_langs,output_langs,test_pairs,max_len_test = prepareData(TEST_DATASET_PATH)
        test_n = len(test_pairs)
        print("Test:sample:", random.choice(test_pairs))
        print(f"Number of Test examples: {test_n}")

        max_len = max(max_len, max(max_len_val, max_len_test)) + 4
        print(max_len)

        # Convert data to tensors and create data loaders
        pairs = makeTensor(input_langs, output_langs, pairs, max_len)
        val_pairs = makeTensor(input_langs, output_langs, val_pairs, max_len)
        test_pairs = makeTensor(input_langs, output_langs, test_pairs, max_len)

        train_loader = DataLoader(dataset = pairs, batch_size=config_defaults[BATCH_SIZE_KEY], shuffle=True)
        val_loader = DataLoader(dataset = val_pairs, batch_size=config_defaults[BATCH_SIZE_KEY], shuffle=True)
        test_loader = DataLoader(dataset = test_pairs, batch_size=config_defaults[BATCH_SIZE_KEY], shuffle=True)

        # Create the encoder and decoder models
        encoder1 = EncoderRNN(
            input_size = input_langs.n_chars,
            embedding_size =  config_defaults[EMBEDDING_SIZE_KEY],
            hidden_size =  config_defaults[HIDDEN_LAYER_KEY],
            num_layers_encoder = config_defaults[ENCODER_LAYER_KEY],
            cell_type = config_defaults[CELL_TYPE_KEY],
            drop_out = config_defaults[DROPOUT_KEY],
            bi_directional = config_defaults[IS_BIDIRECTIONAL_KEY]
            )
        decoder1 = DecoderRNN(
            embedding_size = config_defaults[EMBEDDING_SIZE_KEY], 
            hidden_size = config_defaults[HIDDEN_LAYER_KEY],
            num_layers_decoder = config_defaults[ENCODER_LAYER_KEY],
            cell_type = config_defaults[CELL_TYPE_KEY],
            drop_out = config_defaults[DROPOUT_KEY],
            bi_directional = config_defaults[IS_BIDIRECTIONAL_KEY], 
            output_size = output_langs.n_chars
            )

        if is_gpu:
            encoder1, decoder1 = encoder1.cuda(), decoder1.cuda()

        print("vanilla seq2seqWithoutAttn")
        # Train and evaluate the Seq2SeqWithoutAttn model
        seq2seq(
            encoder = encoder1,
            decoder = decoder1,
            train_loader = train_loader,
            val_loader = val_loader,
            test_loader = test_loader,
            lr = config_defaults[LEARNING_RATE_KEY],
            optimizer = optimizer,
            epochs = config_defaults[EPOCHS_KEY],
            max_length_word = max_len,
            num_layers_enc = config_defaults[ENCODER_LAYER_KEY],
            output_lang = output_langs,
            batch_size = config_defaults[BATCH_SIZE_KEY],
            cell_type = config_defaults[CELL_TYPE_KEY],
            is_wandb = is_wandb
            )


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="dl-assignment-3", entity="cs23m007")
print('sweep_id: ', sweep_id)

